# k.p example of using the Lowdin perturbation

This example requires [semicon](https://gitlab.kwant-project.org/semicon/semicon) to be installed.

It should be as easy as 
```
pip install git+https://gitlab.kwant-project.org/semicon/semicon.git
```

In [ ]:
try:
    import semicon
except ImportError:
    print("Semicon should be installed to run this notebook.")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
import kwant
import semicon

import numpy as np
import scipy.linalg as la

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

# Prepare k.p model and solve it exactly

In [ ]:
widths = [5, 12.5, 5, 5]
gamma_0 = 1.0

grid_spacing = 0.5
shape = lambda site: 0 - grid_spacing / 2 < site.pos[0] < sum(widths)

kpham = semicon.models.foreman('z')

AlSb = semicon.parameters.bulk('lawaetz', 'AlSb', gamma_0, valence_band_offset=.18)
InAs = semicon.parameters.bulk('lawaetz', 'InAs', gamma_0)
GaSb = semicon.parameters.bulk('lawaetz', 'GaSb', gamma_0, valence_band_offset=.56)

params, walls = semicon.parameters.two_deg(
    parameters = [AlSb, InAs, GaSb, AlSb],
    widths = widths,
    grid_spacing=grid_spacing,
    extra_constants=semicon.parameters.constants,
)

In [ ]:
def calc_energy(k):
    p = {'k_x': k, 'k_y': 0}
    mat = syst.hamiltonian_submatrix(params={**p, **params})
    return la.eigvalsh(mat)


template = kwant.continuum.discretize(str(kpham), coords='z', 
                                      grid_spacing=grid_spacing)

syst = kwant.Builder()
syst.fill(template, shape, (0, ))
syst = syst.finalized()

N = len(syst.sites)
momenta = np.linspace(-.3, .3, 81)
energies_exact = np.array([calc_energy(k) for k in momenta])

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(121)
ylims = (400-5, 660)
xlims = (-.3, .3)

window = (.480, .550)

plt.plot(momenta, 1000 * energies_exact, 'k-');
plt.plot(xlims, 1000 * np.array([window, window]))
plt.ylim(*ylims)
plt.xlim(*xlims)



plt.subplot(122)
ylims = (500-5, 530+5)
xlims = (-.3, .3)

window = (.500, .530)

plt.plot(momenta, 1000 * energies_exact, 'k-');
plt.plot(xlims, 1000 * np.array([window, window]))
plt.ylim(*ylims)
plt.xlim(*xlims)

# prepare perturbation basis

In [ ]:
def find_indices(ev, lower, upper):
    return [i for (i, e) in enumerate(ev) if lower < e < upper]

In [ ]:
# for simplicity lets import everything now
from codes.lowdin import *

gens = sympy.symbols(['k_x', 'k_y'])
H0, H1 = prepare_hamiltonian(
    kpham, gens, 'z', grid_spacing, shape, (0,)
)

M = np.diag([1/2, -1/2, 3/2, 1/2, -1/2, -3/2, 1/2, -1/2])
bigM = kwant.operator.Density(H0, M).tocoo().toarray()

## Full diagonalisation: we know everything

In [ ]:
mat0 = H0.hamiltonian_submatrix(params=params)
mat1 = {k: v.hamiltonian_submatrix(params=params) 
        for k, v in H1.items()}

ev, evec = la.eigh(mat0)
indices = find_indices(ev, *window)

U, evs = decouple_basis([bigM, mat0], evec[:, indices])
ev[indices] = evs[-1]

evec[:, indices] = evec[:, indices] @ U
apply_smart_gauge(evec)        

## zero and 1st order

In [ ]:
from codes.lowdin import second_order

In [ ]:
indices = find_indices(ev, *window)

order_0 = (evec[:, indices].T.conj() @ mat0 @ evec[:, indices]).diagonal().real
order_1 = first_order(mat1, evec[:, indices])

In [ ]:
assert np.allclose(order_0, ev[indices])

### 1. We know all states in the system

In [ ]:
%%time
indices = find_indices(ev, *window)

evecA = evec[:, indices]
evecB = np.delete(evec, indices, 1)

exp_only, _ = second_order(mat0, mat1, evecA, evecB)

### 2. We know only states we want to include in the effective model

In [ ]:
%%time
indices = find_indices(ev, *window)
evecA = evec[:, indices]

_, kpm_only = second_order(mat0, mat1, evecA, moments=1000)

### 3. We know states we want to include and some close states

In [ ]:
# prepare subspace (like input from sparse diag)
subspace_slice = find_indices(ev, .400, .660)
energies = ev[subspace_slice]
subspace = evec[:, subspace_slice]
indices = find_indices(energies, *window)

In [ ]:
evecA = subspace[:, indices]
evecB = np.delete(subspace, indices, 1)

exp, kpm = second_order(mat0, mat1, evecA, evecB, moments=1000)

# Look into the effective models

In [ ]:
smp_1 = sympify_perturbation(order_0, components=[order_1, exp_only], decimals=4)
smp_2 = sympify_perturbation(order_0, components=[order_1, kpm_only], decimals=4)
smp_3 = sympify_perturbation(order_0, components=[order_1, exp, kpm], decimals=4)

In [ ]:
smp_1

In [ ]:
smp_2

In [ ]:
smp_3

## compare with exact solution

In [ ]:
def calc_energy(model, k):
    p = {'k_x': k, 'k_y': 0}
    mat = model(**p)
    return la.eigvalsh(mat)


model = kwant.continuum.lambdify(smp_1)
energies_eff_1 = np.array([calc_energy(model, k) for k in momenta])

model = kwant.continuum.lambdify(smp_2)
energies_eff_2 = np.array([calc_energy(model, k) for k in momenta])

model = kwant.continuum.lambdify(smp_3)
energies_eff_3 = np.array([calc_energy(model, k) for k in momenta])

In [ ]:
plt.plot(momenta, 1000 * energies_exact, 'k-');
plt.plot(momenta, 1000 * energies_eff_1, 'r-');
plt.plot(momenta, 1000 * energies_eff_2, 'g--');
plt.plot(momenta, 1000 * energies_eff_3, 'b-.');

plt.plot(xlims, 1000 * np.array([window, window]))
plt.ylim(*ylims)
plt.xlim(*xlims)

# check convergence

In [ ]:
def difference_kpm(kpm):
    M2 = exp_only
    
    assert set(M2) == set(kpm)
    output = 0
    for key, val in M2.items():
        output += la.norm(val - kpm[key])
    return output
    
    
def difference_mixed(exp, kpm):
    M2 = exp_only
    assert set(M2) == set(kpm)
    assert set(M2) == set(exp)
    
    output = 0
    for key, val in M2.items():
        output += la.norm(val - kpm[key] - exp[key])
    return output
    

In [ ]:
moments = range(1000, 10000, 2000)


kpm_ds = []
mixed_ds = []

indices = find_indices(energies, *window)


evecA = subspace[:, indices]
evecB = np.delete(subspace, indices, 1)

for num_moments in moments:
    print(num_moments)
    
    _, pure_kpm = second_order(mat0, mat1, evecA, evecB=None, moments=num_moments)
    exp, kpm = second_order(mat0, mat1, evecA, evecB, moments=num_moments)
    
    kpm_ds.append(difference_kpm(pure_kpm))
    mixed_ds.append(difference_mixed(exp, kpm))
    

In [ ]:
plt.plot(moments, kpm_ds, 'ro-', label='kpm')
plt.plot(moments, mixed_ds, 'bo-', label='mixed')
plt.ylabel('|explicit - kpm|')
plt.xlabel('# moments')
plt.ylim(1e-6, 1)
plt.yscale('log')
plt.grid()
plt.legend()